In [1]:
from pathlib import Path
import os
import sys

sys.path.append(Path(os.path.abspath('')).parent.parent)
os.chdir(Path(os.path.abspath('')).parent.parent)

import pandas as pd
import numpy as np

import config.settings as cset
from config.settings import (labelsize_small, 
                            labelsize_medium, 
                            labelsize_large,
                            focus_countries,
                            focus_countries_names,
                            focus_countries_colors, 
                            focus_scenarios_color_dict,
                            focus_scenarios_labels,
                            cmap_diverging
                            )

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import stats
from scipy.stats import kstest
from scipy.stats import ks_2samp

In [2]:
gmt_ds = pd.read_csv(cset.path_MESMER_char / f'gmt_dataset.csv', index_col = 0)

In [3]:
gdp_df   = pd.read_csv(cset.file_burke).loc[:, ['iso', 'scen', 'run', 'year', 'diff_SSP2', 'SSP2', 'SSP2_clim']]
gdp_df = gdp_df[gdp_df.scen != 'Ref1p5']
gdp_df.loc[gdp_df.scen == 'Refv2', 'scen'] = 'Ref'
gdp_df['scen_run'] = gdp_df['scen'] + '_' + gdp_df['run'].astype(str)

In [ ]:
thrshld = 1.35
thrshld_str = '135'
gmt_char_df = pd.read_csv(cset.path_MESMER_char / f'gmt_characteristics_thsld_{thrshld_str}.csv', index_col = 0)

In [4]:
gmt_ds

,ssp119_1,ssp119_2,ssp119_3,ssp119_4,ssp119_5,ssp119_6,ssp119_7,ssp119_8,ssp119_9,ssp119_10,...,CurPol_91,CurPol_92,CurPol_93,CurPol_94,CurPol_95,CurPol_96,CurPol_97,CurPol_98,CurPol_99,CurPol_100
2015.0,1.1599,0.9586,0.9586,0.9586,1.1652,1.1448,1.1448,0.8845,0.8845,0.9023,...,1.1514,1.2428,1.0327,1.1311,1.1510,1.1653,1.1653,1.2919,1.2919,1.2919
2016.0,1.1710,0.9756,0.9756,0.9756,1.1760,1.1660,1.1660,0.8923,0.8923,0.9088,...,1.1636,1.2678,1.0484,1.1388,1.1705,1.1761,1.1761,1.3078,1.3078,1.3078
2017.0,1.1833,0.9965,0.9965,0.9965,1.1893,1.1893,1.1893,0.9030,0.9030,0.9176,...,1.1776,1.2946,1.0679,1.1499,1.1927,1.1894,1.1894,1.3256,1.3256,1.3256
2018.0,1.1971,1.0190,1.0190,1.0190,1.2042,1.2151,1.2151,0.9165,0.9165,0.9282,...,1.1935,1.3229,1.0895,1.1631,1.2167,1.2044,1.2044,1.3450,1.3450,1.3450
2019.0,1.2111,1.0409,1.0409,1.0409,1.2188,1.2413,1.2413,0.9307,0.9307,0.9387,...,1.2095,1.3514,1.1108,1.1756,1.2406,1.2195,1.2195,1.3640,1.3640,1.3640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096.0,1.0887,1.1331,1.1331,1.1331,1.0575,1.3820,1.3820,0.9295,0.9295,0.8749,...,2.5290,3.5163,2.7254,2.4845,3.0285,2.5750,2.5750,3.1889,3.1889,3.1889
2097.0,1.0827,1.1302,1.1302,1.1302,1.0517,1.3765,1.3765,0.9266,0.9266,0.8707,...,2.5492,3.5501,2.7501,2.5037,3.0560,2.5948,2.5948,3.2181,3.2181,3.2181
2098.0,1.0766,1.1272,1.1272,1.1272,1.0458,1.3710,1.3710,0.9236,0.9236,0.8665,...,2.5695,3.5840,2.7748,2.5229,3.0835,2.6145,2.6145,3.2474,3.2474,3.2474
2099.0,1.0704,1.1242,1.1242,1.1242,1.0399,1.3654,1.3654,0.9206,0.9206,0.8621,...,2.5898,3.6180,2.7995,2.5422,3.1111,2.6343,2.6343,3.2767,3.2767,3.2767
